In [1]:
%pip install h5py
%pip install tables

# Setup/Needed Imports

from google.cloud import bigquery
from IPython.display import display, HTML
from google.cloud import storage

import pandas as pd
import numpy as np

from shared_libs import data_prep as dp
from shared_libs import model_train as mt

BUCKET_NAME = 'python-testing-re'

Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 4.6MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 163kB 21.8MB/s eta 0:00:01
  Created wheel for tables: filename=tables-3.6.1-cp35-cp35m-linux_x86_64.whl size=8754007 sha256=63fe4f703da82d62a7c4d05c5540263e4b23db43dd1f11003572bb736e5e13f8
  Stored in directory: /home/jupyter/.cache/pip/wheels/3a/a6/49/e6cb0ce92e7285472e373323b27499a10cffe247f380bf5b6e
Successfully built tables
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/numexpr-2.7.1.dist-info'
Consider using the `--user` option or check the permissions.

Note: you may need to restart the kernel to use updated packages.


ImportError: No module named 'shared_libs'